In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import jellyfish
import gensim
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [4]:
#he competition dataset contains text from works of fiction written by spooky authors 
#of the public domain: Edgar Allan Poe, HP Lovecraft and Mary Shelley. 
#The data was prepared by chunking larger texts into sentences using CoreNLP's 
#MaxEnt sentence tokenizer, so you may notice the odd non-sentence here and there. 
#Your objective is to accurately identify the author of the sentences in the test set.

In [5]:
# From kaggle task: https://www.kaggle.com/c/spooky-author-identification/data
test = pd.read_csv("/Users/janpintera/Dropbox/Spooky datasets/spooky_test.csv")
train = pd.read_csv("/Users/janpintera/Dropbox/Spooky datasets/train.csv")

In [6]:
X = train['text']
Y = train['author']
X_train, X_val , y_train, y_val = train_test_split(X, Y, test_size=0.33, random_state=42)
X_target = test['text']

In [7]:
train.groupby('author').agg('count')

,id,text
author,,
EAP,7900,7900
HPL,5635,5635
MWS,6044,6044


In [8]:
Tvectorizer = TfidfVectorizer(stop_words = 'english')
dicT = Tvectorizer.fit(X_train)
feat_names = dicT.get_feature_names()

In [10]:
Xm = Tvectorizer.transform(X_train)
Xt = Tvectorizer.transform(X_val)

In [11]:
Xm

<13117x21269 sparse matrix of type '<class 'numpy.float64'>'
	with 147953 stored elements in Compressed Sparse Row format>

In [12]:
#Bernoulli Naive Bayes Classifier:
clf = BernoulliNB()
clf.fit(Xm, y_train)
prediction = clf.predict(Xt)

In [13]:
prediction

array(['EAP', 'MWS', 'MWS', ..., 'MWS', 'EAP', 'EAP'],
      dtype='<U3')

In [14]:
confusion_matrix(prediction,y_val)

array([[2355,  470,  423],
       [  64, 1250,   48],
       [ 168,  132, 1552]])

In [15]:
# Multinomial Naive Bayes Classifier:
mNB = MultinomialNB()
mNB.fit(Xm, y_train)
predict_mNB = mNB.predict(Xt)

In [16]:
confusion_matrix(predict_mNB,y_val)

array([[2265,  436,  352],
       [ 102, 1289,   66],
       [ 220,  127, 1605]])

In [19]:
# Latent dirichlet allocation
lda = LatentDirichletAllocation(n_components=3, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [20]:
lda.fit(Xm)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_components=3, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [37]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [36]:
for topic_idx, topic in enumerate(lda.components_):
 print(topic_idx,topic)

0 [ 0.35642702  0.34153979  0.9856561  ...,  0.33477412  0.53144622
  0.33421872]
1 [ 0.47146981  0.93369331  0.37157729 ...,  0.45983054  0.34216951
  0.55384639]
2 [ 0.33548261  0.50104363  3.2791923  ...,  0.35183447  0.33566191
  0.33486895]


In [39]:
print_top_words(lda, feat_names, 10)

Topic #0: did said man old life saw time like know long
Topic #1: room head feet remained doubt hand fact having left entered
Topic #2: mr door returned arrived second chamber followed opened house simple

